# Quickstart: Agentic retrieval in Azure AI Search

Use this notebook to get started with [agentic retrieval](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept) in Azure AI Search, which integrates an Azure OpenAI chat completion model to process queries, retrieve relevant content from indexed documents, and generate natural-language answers.

Steps in this notebook include:

1. Creating and loading an `earth-at-night` search index.

1. Creating an `earth-knowledge-source` that targets your index.

1. Creating an `earth-knowledge-agent` that targets your knowledge source and an LLM for query planning and answer synthesis.

1. Using the agent to fetch, rank, and synthesize relevant information from the index.

This notebook provides a high-level demonstration of agentic retrieval. For more detailed guidance, see [Quickstart: Run agentic retrieval in Azure AI Search](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval).

## Prerequisites

+ An [Azure AI Search service](https://learn.microsoft.com/azure/search/search-create-service-portal) on the Basic tier or higher with [semantic ranker enabled](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable).

+ An [Azure AI Foundry project](https://learn.microsoft.com/azure/ai-foundry/how-to/create-projects) and Azure AI Foundry resource. When you create a project, the resource is automatically created.

+ A [supported chat completion model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models). This sample uses `gpt-4.1-mini`.

+ A text embedding model. This sample uses `text-embedding-3-large`.

+ [Visual Studio Code](https://code.visualstudio.com/download) with the [Python extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python) and [Jupyter package](https://pypi.org/project/jupyter/).

## Configure access

This notebook assumes that you're using Microsoft Entra ID for authentication and role assignments for authorization.

To configure role-based access:

1. Sign in to the [Azure portal](https://portal.azure.com).

1. On your Azure AI Search service:

    1. [Enable role-based access](https://learn.microsoft.com/azure/search/search-security-enable-roles).
    
    1. [Create a system-assigned managed identity](https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources#create-a-system-managed-identity).
    
    1. [Assign the following roles](https://learn.microsoft.com/azure/search/search-security-rbac#how-to-assign-roles-in-the-azure-portal) to yourself.
    
       + **Search Service Contributor**
    
       + **Search Index Data Contributor**
    
       + **Search Index Data Reader**

1. On your Azure AI Foundry resource, assign **Cognitive Services User** to the managed identity of your search service.

## Set up connections

The `sample.env` file contains environment variables for connections to Azure AI Search and Azure OpenAI in Azure AI Foundry. Agentic retrieval requires these connections for document retrieval, query planning, and query execution.

To set up the connections:

1. Sign in to the [Azure portal](https://portal.azure.com).

1. Get the endpoints for Azure AI Search (`https://your-search-service.search.windows.net`) and Azure OpenAI in Azure AI Foundry (`https://your-foundry-resource.openai.azure.com`).

1. Save the `sample.env` file as `.env` on your local system.

1. Update the `.env` file with the retrieved endpoints.

## Create a virtual environment

The `requirements.txt` file contains the dependencies for this notebook. You can use a virtual environment to install these dependencies in isolation.

To create a virtual environment:

1. In Visual Studio Code, open the folder that contains `quickstart-agentic-retrieval.ipynb`.

1. Press **Ctrl**+**Shift**+**P** to open the command palette.

1. Search for **Python: Create Environment**, and then select **Venv**.

1. Select a Python installation. We tested this notebook on Python 3.13.7.

1. Select `requirements.txt` for the dependencies.

Creating the virtual environment can take several minutes. When the environment is ready, proceed to the next step.

## Install packages and load connections

This step installs the packages for this notebook and establishes connections to Azure AI Search and Azure OpenAI in Azure AI Foundry.

In [23]:
! pip install -r requirements.txt --quiet

In [24]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

# Take environment variables from .env
load_dotenv(override=True)

# This notebook uses the following variables from your .env file
search_endpoint = os.environ["SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
aoai_endpoint = os.environ["AOAI_ENDPOINT"]
aoai_embedding_model = os.environ["AOAI_EMBEDDING_MODEL"]
aoai_embedding_deployment = os.environ["AOAI_EMBEDDING_DEPLOYMENT"]
aoai_gpt_model = os.environ["AOAI_GPT_MODEL"]
aoai_gpt_deployment = os.environ["AOAI_GPT_DEPLOYMENT"]
index_name = os.environ["INDEX_NAME"]
knowledge_source_name = os.environ["KNOWLEDGE_SOURCE_NAME"]
knowledge_agent_name = os.environ["KNOWLEDGE_AGENT_NAME"]
search_api_version = os.environ["SEARCH_API_VERSION"]

## Create a search index

This step creates an index that contains plain text and vector content. You can use an existing index, but it must meet the criteria for [agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schema requirement is a semantic configuration with a `default_configuration_name`.

In [27]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient
from openai import AzureOpenAI
from azure.identity import get_bearer_token_provider

azure_openai_token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=aoai_endpoint,
                    deployment_name=aoai_embedding_deployment,
                    model_name=aoai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully.")

Index 'earth-at-night' created or updated successfully.


## Upload sample documents

This notebook uses data from NASA's Earth at Night e-book. The data is retrieved from the [azure-search-sample-data](https://github.com/Azure-Samples/azure-search-sample-data) repository on GitHub and passed to the search client for indexing.

In [28]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=search_endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}' successfully.")

Documents uploaded to index 'earth-at-night' successfully.


## Create a knowledge source

This step creates a knowledge source that targets the index you previously created. In the next step, you create a knowledge agent that uses the knowledge source to orchestrate agentic retrieval.

In [29]:
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters
from azure.search.documents.indexes import SearchIndexClient

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_select="id,page_chunk,page_number",
    ),
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks, api_version=search_api_version)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

Knowledge source 'earth-knowledge-source' created or updated successfully.


## Create a knowledge agent

This step creates a knowledge agent, which acts as a wrapper for your knowledge source and LLM deployment.

`EXTRACTIVE_DATA` is the default modality and returns content from your knowledge sources without generative alteration. However, this quickstart uses the `ANSWER_SYNTHESIS` modality for LLM-generated answers that cite the retrieved content.

In [30]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeAgentOutputConfiguration, KnowledgeAgentOutputConfigurationModality
from azure.search.documents.indexes import SearchIndexClient

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=aoai_endpoint,
    deployment_name=aoai_gpt_deployment,
    model_name=aoai_gpt_model,
)

output_cfg = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.ANSWER_SYNTHESIS,
    include_activity=True,
)

agent = KnowledgeAgent(
    name=knowledge_agent_name,
    models=[KnowledgeAgentAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source_name,
            reranker_threshold=2.5,
        )
    ],
    output_configuration=output_cfg,
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_agent(agent, api_version=search_api_version)
print(f"Knowledge agent '{knowledge_agent_name}' created or updated successfully.")

Knowledge agent 'earth-knowledge-agent' created or updated successfully.


## Set up messages

Messages are the input for the retrieval route and contain the conversation history. Each message includes a `role` that indicates its origin, such as `system` or `user`, and `content` in natural language. The LLM you use determines which roles are valid.

In [31]:
instructions = """
A Q&A agent that can answer questions about the Earth at night.
If you don't have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

## Use agentic retrieval to fetch results

This step runs the agentic retrieval pipeline to produce a grounded, citation-backed answer. Given the conversation history and retrieval parameters, your knowledge agent:

1. Analyzes the entire conversation to infer the user's information need.

1. Decomposes the compound query into focused subqueries.

1. Executes the subqueries concurrently against your knowledge source.

1. Uses semantic ranker to rerank and filter the results.

1. Synthesizes the top results into a natural-language answer.

In [33]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, SearchIndexKnowledgeSourceParams

agent_client = KnowledgeAgentRetrievalClient(endpoint=search_endpoint, agent_name=knowledge_agent_name, credential=credential)
query_1 = """
    Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
    Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """

messages.append({
    "role": "user",
    "content": query_1
})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            kind="searchIndex"
        )
    ]
)

result = agent_client.retrieve(retrieval_request=req, api_version=search_api_version)
print(f"Retrieved content from '{knowledge_source_name}' successfully.")

Retrieved content from 'earth-knowledge-source' successfully.


### Review the retrieval response, activity, and results

Because your knowledge agent is configured for answer synthesis, the retrieval response contains the following values:

+ `response_content`: An LLM-generated answer to the query that cites the retrieved documents.

+ `activity_content`: Detailed planning and execution information, including subqueries, reranking decisions, and intermediate steps.

+ `references_content`: Source documents and chunks that contributed to the answer.

**Tip:** Retrieval parameters, such as reranker thresholds and knowledge source parameters, influence how aggressively your agent reranks and which sources it queries. Inspect the activity and references to validate grounding and build traceable citations.

In [7]:
response_contents = []
activity_contents = []
references_contents = []

In [8]:
import json

# Build simple string values for response_content, activity_content, and references_content

# Responses -> Concatenate text/value fields from all response contents
response_parts = []
if getattr(result, "response", None):
    for resp in result.response:
        for content in getattr(resp, "content", []):
            text = getattr(content, "text", None) or getattr(content, "value", None) or str(content)
            response_parts.append(text)
response_content = "\n\n".join(response_parts) if response_parts else "No response found on 'result'"

response_contents.append(response_content)

# Print the three string values
print("response_content:\n", response_content, "\n")

response_content:
 The Phoenix metropolitan area exhibits a sharply visible nighttime street grid from space due to its regular grid layout of city blocks and streets, which is typical of many large urban areas in the central and western United States. This grid pattern is especially evident at night because of the street lighting, which illuminates the streets and major thoroughfares such as Grand Avenue, creating a clear and extensive network of lights visible from low-Earth orbit. The urban grid encourages outward growth along city borders, supported by widespread personal automobile use, resulting in a large metropolitan area with interconnected municipalities linked by surface streets and freeways. In contrast, large stretches of interstate highways between Midwestern cities remain comparatively dim because they do not have the dense, regularly spaced street lighting and urban development that characterize the Phoenix metropolitan area. Additionally, the Phoenix area includes brig

In [9]:
messages.append({
    "role": "assistant",
    "content": response_content
})

In [10]:
# Activity -> JSON string of activity as list of dicts
if getattr(result, "activity", None):
    activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
else:
    activity_content = "No activity found on 'result'"
    
activity_contents.append(activity_content)
print("activity_content:\n", activity_content, "\n")

activity_content:
 [
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 5743,
    "input_tokens": 2071,
    "output_tokens": 179
  },
  {
    "id": 1,
    "type": "searchIndex",
    "elapsed_ms": 596,
    "knowledge_source_name": "earth-knowledge-source",
    "query_time": "2025-09-05T15:41:06.930Z",
    "count": 0,
    "search_index_arguments": {
      "search": "Reasons for larger December brightening in suburban belts compared to urban cores despite higher downtown light levels"
    }
  },
  {
    "id": 2,
    "type": "searchIndex",
    "elapsed_ms": 329,
    "knowledge_source_name": "earth-knowledge-source",
    "query_time": "2025-09-05T15:41:07.260Z",
    "count": 2,
    "search_index_arguments": {
      "search": "Factors making Phoenix nighttime street grid sharply visible from space"
    }
  },
  {
    "id": 3,
    "type": "searchIndex",
    "elapsed_ms": 394,
    "knowledge_source_name": "earth-knowledge-source",
    "query_time": "2025-09-05T15:41:07.655Z",

In [11]:
# References -> JSON string of references as list of dicts
if getattr(result, "references", None):
    references_content = json.dumps([r.as_dict() for r in result.references], indent=2)
else:
    references_content = "No references found on 'result'"
    
references_contents.append(references_content)
print("references_content:\n", references_content)

references_content:
 [
  {
    "type": "searchIndex",
    "id": "0",
    "activity_source": 2,
    "reranker_score": 2.6642752,
    "doc_key": "earth_at_night_508_page_104_verbalized"
  },
  {
    "type": "searchIndex",
    "id": "1",
    "activity_source": 2,
    "reranker_score": 2.5905457,
    "doc_key": "earth_at_night_508_page_105_verbalized"
  }
]


## Continue the conversation

This step continues the conversation with your knowledge agent, building upon the previous messages and queries to retrieve relevant information from your knowledge source.

In [ ]:
query_2 = "How do I find lava at night?"
messages.append({
    "role": "user",
    "content": query_2
})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            kind="searchIndex"
        )
    ]
)

result = agent_client.retrieve(retrieval_request=req, api_version=search_api_version)
print(f"Retrieved content from '{knowledge_source_name}' successfully.")

Retrieved content from 'knowledge_source_name' successfully.


### Review the retrieval response, activity, and results

In [13]:
import json

# Build simple string values for response_content, activity_content, and references_content

# Responses -> Concatenate text/value fields from all response contents
response_parts = []
if getattr(result, "response", None):
    for resp in result.response:
        for content in getattr(resp, "content", []):
            text = getattr(content, "text", None) or getattr(content, "value", None) or str(content)
            response_parts.append(text)
response_content = "\n\n".join(response_parts) if response_parts else "No response found on 'result'"

response_contents.append(response_content)

# Print the three string values
print("response_content:\n", response_content, "\n")

response_content:
 Lava can be found at night by using satellite imagery that captures thermal, shortwave infrared, and near-infrared wavelengths, which highlight very hot lava as bright white and cooling lava as red. For example, Landsat 8 imagery has been used to track lava flows at Hawaii's Kilauea volcano during nighttime, showing the progression of lava flows with these thermal signatures [ref_id:0]. Similarly, the VIIRS Day/Night Band (DNB) on the Suomi NPP satellite can capture nighttime images of lava flows, as demonstrated by images of Mount Etna's lava glow in Italy [ref_id:2][ref_id:5]. These instruments use faint light sources such as moonlight to illuminate the Earth's surface at night, allowing the detection of glowing lava flows. Combining thermal data with nightlight imagery provides a clear view of volcanic activity after dark [ref_id:1]. 



In [14]:
# Activity -> JSON string of activity as list of dicts
if getattr(result, "activity", None):
    activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
else:
    activity_content = "No activity found on 'result'"
    
activity_contents.append(activity_content)
print("activity_content:\n", activity_content, "\n")

activity_content:
 [
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 1848,
    "input_tokens": 2351,
    "output_tokens": 83
  },
  {
    "id": 1,
    "type": "searchIndex",
    "elapsed_ms": 521,
    "knowledge_source_name": "earth-knowledge-source",
    "query_time": "2025-09-05T15:41:31.525Z",
    "count": 6,
    "search_index_arguments": {
      "search": "How to locate lava flows at night"
    }
  },
  {
    "id": 2,
    "type": "searchIndex",
    "elapsed_ms": 271,
    "knowledge_source_name": "earth-knowledge-source",
    "query_time": "2025-09-05T15:41:31.797Z",
    "count": 2,
    "search_index_arguments": {
      "search": "Methods for detecting lava at night"
    }
  },
  {
    "id": 3,
    "type": "searchIndex",
    "elapsed_ms": 315,
    "knowledge_source_name": "earth-knowledge-source",
    "query_time": "2025-09-05T15:41:32.112Z",
    "count": 0,
    "search_index_arguments": {
      "search": "Safety tips for finding lava at night"
    }
  },
  {
  

In [15]:
# References -> JSON string of references as list of dicts
if getattr(result, "references", None):
    references_content = json.dumps([r.as_dict() for r in result.references], indent=2)
else:
    references_content = "No references found on 'result'"
    
references_contents.append(references_content)
print("references_content:\n", references_content)

references_content:
 [
  {
    "type": "searchIndex",
    "id": "0",
    "activity_source": 1,
    "reranker_score": 2.779123,
    "doc_key": "earth_at_night_508_page_60_verbalized"
  },
  {
    "type": "searchIndex",
    "id": "2",
    "activity_source": 1,
    "reranker_score": 2.7684891,
    "doc_key": "earth_at_night_508_page_64_verbalized"
  },
  {
    "type": "searchIndex",
    "id": "1",
    "activity_source": 2,
    "reranker_score": 2.6940506,
    "doc_key": "earth_at_night_508_page_46_verbalized"
  },
  {
    "type": "searchIndex",
    "id": "4",
    "activity_source": 1,
    "reranker_score": 2.6189032,
    "doc_key": "earth_at_night_508_page_66_verbalized"
  },
  {
    "type": "searchIndex",
    "id": "3",
    "activity_source": 2,
    "reranker_score": 2.5664418,
    "doc_key": "earth_at_night_508_page_44_verbalized"
  },
  {
    "type": "searchIndex",
    "id": "5",
    "activity_source": 1,
    "reranker_score": 2.5382614,
    "doc_key": "earth_at_night_508_page_65_verba

## Clean up objects and resources

If you no longer need Azure AI Search or Azure AI Foundry, delete the resources from your Azure subscription. You can also start over by deleting individual objects.

### Delete the knowledge agent

In [17]:
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.delete_agent(knowledge_agent_name)
print(f"Knowledge agent '{knowledge_agent_name}' deleted successfully.")

Knowledge agent 'earth-knowledge-agent' deleted successfully.


### Delete the knowledge source

In [ ]:
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.delete_knowledge_source(knowledge_source=knowledge_source_name) # This is new feature in 2025-08-01-Preview api version
print(f"Knowledge source '{knowledge_source_name}' deleted successfully.")

Knowledge source 'earth-knowledge-source' deleted successfully.


### Delete the search index

In [21]:
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.delete_index(index_name)
print(f"Index '{index_name}' deleted successfully.")

Index 'earth-at-night' deleted successfully.
